In [1]:
SQLCommand = ("INSERT INTO table_name ([Field 1], [Field 2], [Field 3]) VALUES ('2014-12-31', 1, 2);")
cursor.execute(SQLCommand)
Connection.commit()

NameError: name 'cursor' is not defined

##ticketPrint to AZURE sql

In [3]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc


In [4]:
server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [8]:
tradeid = '7'                                 ##'yyyymmdd + secuencial number'--
timestamp =  '10:45'                          ## 
side =      'bid'                             ## bid - ask
ncliente = '116244'                           ##de 6 digitos nada mas--
descliente = 'Capitales Monedas y Valores'    ##razon social o nombre de cliente--
ccycom = 'USD'                                ##Moneda de comprar--
montocom = '3000'                             ##monto a comprar, expresado en moneda de compra--
px_sp = '605'                                 ##precio sport --
ccyven = 'CRC'                                ##moneda a vender--
montoven = '1815000'                          ##monto a vender --
estado = 'aplicada'                           ##aplicada, pendiente, anulada--
ejecutivo = 'EGROSSO'                         ##ejecutivo que atiende la cuenta--
action = 'add'                                ##agrega, cancela, reinstate--
tradedate = '20210213'                        ##date
valuedate = '20210213'                        ##date 
monex_pxlast = '612.61'                       ##last traded price in Monex--
monex_pp = '612.64'                           ##last update weighted average Monex--
dealer = 'EstebanC'                           ##trader that inputs the transaction-- 
px_cover = '606'                              ##internal price split--
productotipo = 'SPOT'                         ##spot o forward-- 
segmento = 'corp'                             ##segmento al que pertenece el cliente--
solicitud = '400113'                          ##numero de solicitud de sama--


SQLCommand = ('INSERT INTO [dbo].[fxblotter.t] ([tradeid], [timestamp], [side], [ncliente], [descliente], [ccycom], [montocom], [px_sp], [ccyven], [montoven], [estado], [ejecutivo], [action], [tradedate], [valuedate], [monex_pxlast], [monex_pp], [dealer], [px_cover], [productotipo], [segmento], [solicitud]) VALUES ( '+timestamp+', '+side+', '+ncliente+', '+descliente+', '+ccycom+', '+montocom+', '+px_sp+', '+ccyven+', '+montoven+', '+estado+', '+ejecutivo+', '+action+', '+tradedate+', '+valuedate+', '+monex_pxlast+', '+monex_pp+', '+dealer+', '+px_cover+', '+productotipo+', '+segmento+', '+solicitud+');')
cursor.execute(SQLCommand)
Connection.commit()

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near ':'. (102) (SQLExecDirectW)")

In [2]:
INSERT INTO [dbo].[fxblotter.t] ([tradeid], [timestamp], [side], [ncliente], [descliente], [ccycom], [montocom], [px_sp], [ccyven], [montoven], [estado], [ejecutivo], 
                                  [action], [tradedate], [valuedate], [monex_pxlast], [monex_pp], [dealer], [px_cover], [productotipo], [segmento], [solicitud]        
)
    
VALUES (--[tradeid] its 'yyyymmdd + secuencial number'--
        '7',
        --[timestamp]--- 
        '10:45',
        --[side]-- 
        'bid',
         --[ncliente] de 6 digitos nada mas--
         '116244',
         --[descliente] razon social o nombre de cliente--
         'Capitales Monedas y Valores',
         --[ccycom] Moneda de comprar--
         'USD',
         --[montocom] monto a comprar, expresado en moneda de compra--
         '3000',
         --[px_sp] precio sport --
         '605',
         --[ccyven] moneda a vender--
         'CRC',
         --[montoven] monto a vender --
         '1815000',
         --[estado] aplicada, pendiente, anulada--
         'aplicada',
         --[ejecutivo] ejecutivo que atiende la cuenta--
         'EGROSSO',
         --[action]agrega, cancela, reinstate--
         'add', 
         --[tradedate] date--
         '20210213',
         --[valuedate] date-- 
         '20210213', 
         --[monex_pxlast] last traded price in Monex--
         '6612.61', 
         --[monex_pp] last update weighted average Monex--
         '6612.64', 
         --[dealer] trader that inputs the transaction--
         'EstebanC', 
         --[px_cover] internal price split--
         '606', 
         --[productotipo] spot o forward--
         'SPOT', 
         --[segmento] segmento al que pertenece el cliente--
         'corp', 
         --[solicitud] numero de solicitud de sama--
         '400113'
);

SyntaxError: invalid syntax (<ipython-input-2-83d9399b05b0>, line 1)